In [1]:
%pip install torchvision

Note: you may need to restart the kernel to use updated packages.


In [2]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize pixel values
])

# Download and load the CIFAR-10 training data
trainset = datasets.CIFAR10(root='./data', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)


100%|██████████| 170M/170M [00:55<00:00, 3.05MB/s] 


In [3]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),   # Randomly flip images horizontally
    transforms.RandomCrop(32, padding=4),# Randomly crop with padding
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalize
])

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define a simple neural network
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1) # Flatten the image
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

# Initialize the model, loss function, and optimizer
model = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.8777589264428218
Epoch 2, Loss: 1.6596495472561672
Epoch 3, Loss: 1.5771942989295706
Epoch 4, Loss: 1.522656711623492
Epoch 5, Loss: 1.4796660149188907
Epoch 6, Loss: 1.4421239431251955
Epoch 7, Loss: 1.4086235859205045
Epoch 8, Loss: 1.3777961051067733
Epoch 9, Loss: 1.3475356863435273
Epoch 10, Loss: 1.3206031417755215


In [5]:
# Validation data transformations (only normalization, no augmentation)
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize pixel values
])

# Download and load the CIFAR-10 validation data
valset = datasets.CIFAR10(root='./data', download=True, train=False, transform=val_transform)
valloader = DataLoader(valset, batch_size=64, shuffle=False)


In [6]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")


Validation Loss: 1.3917154954497222, Validation Accuracy: 50.96%


In [7]:
import torch.nn.functional as F

class SimpleNNWithDropout(nn.Module):
    def __init__(self):
        super(SimpleNNWithDropout, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.dropout = nn.Dropout(0.5)  # Drop 50% of neurons
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)  # Apply dropout
        return self.fc2(x)


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim



# Initialize the model, loss function, and optimizer
model = SimpleNNWithDropout()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.913388651045387
Epoch 2, Loss: 1.7309877087393075
Epoch 3, Loss: 1.6656557557832858
Epoch 4, Loss: 1.6257933321816231
Epoch 5, Loss: 1.5936748612567286
Epoch 6, Loss: 1.5642766036340952
Epoch 7, Loss: 1.5452047135214062
Epoch 8, Loss: 1.5217729191036176
Epoch 9, Loss: 1.5058371708216265
Epoch 10, Loss: 1.4930074530489303


In [9]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")

Validation Loss: 1.4387596207819167, Validation Accuracy: 49.81%


In [10]:
class SimpleNNWithBatchNorm(nn.Module):
    def __init__(self):
        super(SimpleNNWithBatchNorm, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 256)
        self.batch_norm = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 10)
        
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = F.relu(self.batch_norm(self.fc1(x)))  # Apply batch normalization
        return self.fc2(x)


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim



# Initialize the model, loss function, and optimizer
model = SimpleNNWithBatchNorm()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Training loop
for epoch in range(10): # 10 epochs
    running_loss = 0.0
    for images, labels in trainloader:
        optimizer.zero_grad()       # Zero the gradients
        
        outputs = model(images)     # Forward pass
        loss = criterion(outputs, labels) # Compute loss
        loss.backward()             # Backward pass
        optimizer.step()            # Update weights
        
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")


Epoch 1, Loss: 1.7114622403898507
Epoch 2, Loss: 1.5291721750708187
Epoch 3, Loss: 1.448771873093627
Epoch 4, Loss: 1.3881968735429027
Epoch 5, Loss: 1.3388333471534808
Epoch 6, Loss: 1.298935681657718
Epoch 7, Loss: 1.2593121647529895
Epoch 8, Loss: 1.22461650164231
Epoch 9, Loss: 1.196194169085349
Epoch 10, Loss: 1.164132782641579


In [12]:
# Validation loop
val_loss = 0.0
correct = 0
total = 0

# Switch the model to evaluation mode
model.eval()

with torch.no_grad():  # No need to track gradients during validation
    for images, labels in valloader:
        outputs = model(images)
        loss = criterion(outputs, labels)  # Compute the loss on the validation set
        val_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)  # Get the predicted class with the highest score
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate average loss and accuracy
avg_val_loss = val_loss / len(valloader)
accuracy = 100 * correct / total

print(f"Validation Loss: {avg_val_loss}, Validation Accuracy: {accuracy}%")

Validation Loss: 1.4024349390321476, Validation Accuracy: 50.88%
